<a href="https://www.kaggle.com/code/antonasnitsky/naya-ds2022-calassification-project?scriptVersionId=117212494" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from sklearn import metrics
import math


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exercise-recognition/train.csv


# Import Libraries

In [2]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

Let's load the dataset

# EDA

In [3]:
df=pd.read_csv('/kaggle/input/exercise-recognition/train.csv')
df.head()

,pose_id,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0,squats_up,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,1,situp_down,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,2,situp_down,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,3,jumping_jacks_up,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,4,jumping_jacks_down,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677


Let's  measure null values.

In [4]:
df.isnull().sum().sort_values(ascending=False)

pose_id               0
z_right_index_1       0
x_right_hip           0
z_left_hip            0
y_left_hip            0
                     ..
z_mouth_left          0
y_mouth_left          0
x_mouth_left          0
z_right_ear           0
z_right_foot_index    0
Length: 101, dtype: int64

Let's see our labels

In [5]:
df['pose'].value_counts()

jumping_jacks_down    151
jumping_jacks_up      145
pullups_down          123
pushups_up            115
squats_up             111
pullups_up            108
squats_down           101
situp_down             82
pushups_down           82
situp_up               79
Name: pose, dtype: int64

# Split Dataset


In [6]:
df.dropna()
X = df.drop(['pose_id','pose'], axis=1)
y = df['pose']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)

print(X_train.shape)

y_train.shape


(767, 99)


(767,)

# Transformations

Let's create more common label. We can clearly see that some poses are the same exercise' just in opposite directions.

**Creating a transformer for the features**

In [7]:
class XYZ_Data_Transformer():
    data = pd.DataFrame()
    scaler = StandardScaler()
    
    def __init__(self):
        pass
    
    def fit(self, data):
        self.data = data
        return self
    
    def __create_regions_list(self, cords):
        cords_name = []
        for cord in cords:
            cord_name = cord[2:]
            if cord_name not in cords_name: #creating list of regions, without x_, y_ and z_
                cords_name.append(cord_name)
        return cords_name

    def __get_xyz_cords_from_row(self, row, region):
        x = row['x_' + region]
        y = row['y_' + region]
        z = row['z_' + region]
        return x,y,z

    def __convert_data_to_xyz_dict(self):
        new_data = []
        data_cols = self.data.columns.tolist()
        regions_list = self.__create_regions_list(data_cols)
        for index, row in self.data.iterrows():
            row_xyz = self.__get_magnitude(row, regions_list) #converting x,y,z coordinates to one index
            new_data.append(row_xyz)
            
        print("Data shape before")
        print(self.data.shape)
        
        magnitudes = pd.DataFrame.from_dict(new_data)
        print("Magnitutes shape")
        print(magnitudes.shape)
        self.data = pd.concat([self.data.reset_index(drop=True), magnitudes.reset_index(drop=True)], axis=1).fillna(0)
        
        print("SHape after concat")
        print(self.data.shape)
        pass

    def __get_magnitude(self, row, regions_list):
        xyz_cords = {}
        for region in regions_list:
            x, y, z = self.__get_xyz_cords_from_row(row, region)
            vector = np.array([x,y,z])
            xyz_cords[region] = math.sqrt(sum(pow(element, 2) for element in vector))
        return xyz_cords

    def transform(self):
        self.__convert_data_to_xyz_dict() #converting each x,y,z columns to one column
        return  pd.DataFrame(self.scaler.fit_transform(self.data),columns = self.data.columns) #scaling the data

**Creating a transformer for the label**

In [8]:
class Pose_Names_Transformer():
    data = pd.DataFrame()
    
    def __init__(self):
        pass
    
    def fit(self, data):
        self.data = data
        return self
    
    def __change_pose_name_to_int(self): #converting poses names into numbers
        poses = self.data.pose.unique()
        poses_dict = {}
        for i in range(len(poses)):
            poses_dict[poses[i]] = i
        self.data = self.data.replace({"pose": poses_dict})
        pass

    def transform(self):
        return change_pose_name_to_int(self.data[['pose']])


**Functions for the transformer**

In [9]:
#def convert_data_to_xyz_dict(data):
#    new_data = []
#    data_cols = data.columns.tolist()
#    regions_list = create_regions_list(data_cols)
#    for index, row in data.iterrows():
#        row_xyz = create_indexs_of_xyz_coor1(row, regions_list) #converting x,y,z coordinates to one index
#        new_data.append(row_xyz)
#    return new_data

In [10]:
#def create_regions_list(cords):
#    cords_name = []
#    for cord in cords:
#        cord_name = cord[2:]
#        if cord_name not in cords_name: #creating list of regions, without x_, y_ and z_
#            cords_name.append(cord_name)
#    return cords_name

In [11]:
#def create_indexs_of_xyz_coor(row, regions_list):
#    xyz_cords = {}
#    for region in regions_list:
#        x, y, z = get_xyz_cords_from_row(row, region)
#        xyz_cords[region] = xyz_to_index(x,y,z)
#    return xyz_cords

In [12]:
#def create_indexs_of_xyz_coor1(row, regions_list):
#    xyz_cords = {}
#    for region in regions_list:
#        x, y, z = get_xyz_cords_from_row(row, region)
#        vector = np.array([x,y,z])
#        xyz_cords[region] = magnitude(vector)
#    return xyz_cords

In [13]:
#def xyz_to_index(x,y,z):
#    return ((x * 100) + y) * 100 + z

In [14]:
#def get_xyz_cords_from_row(row, region):
#    x = row['x_' + region]
#    y = row['y_' + region]
#    z = row['z_' + region]
#    return x,y,z

In [15]:
#def change_pose_name_to_int(data): #converting poses names into numbers
#    poses = data.pose.unique()
#    poses_dict = {}
#    for i in range(len(poses)):
#        poses_dict[poses[i]] = i
#    new_data = data.replace({"pose": poses_dict})
#    return new_data

In [16]:
#def magnitude(vector):
#    return math.sqrt(sum(pow(element, 2) for element in vector))

In [17]:
X_train.head()

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
827,-1.438639,46.420020,9.177941,-0.396456,48.602740,4.358058,0.241232,48.654250,4.364322,0.876346,...,70.292404,-2.059812,-8.752794,79.396530,-6.967450,-14.974481,73.159960,-6.945509,-9.229867,85.010580
499,-30.114735,52.610325,-118.979070,-33.722590,47.545303,-121.604225,-35.987785,40.527145,-121.620860,-36.384780,...,90.788760,33.030914,-31.291740,107.379684,42.682987,-29.043562,85.841446,40.658980,-25.659513,106.590096
866,-0.351658,-57.733524,-39.715270,0.806687,-59.820293,-37.092990,1.452499,-59.624653,-37.107410,2.081463,...,34.222786,-2.374286,58.859890,34.025920,3.895650,66.325226,17.748278,-1.605374,65.596240,16.257936
345,4.644966,-52.943260,25.799000,4.303432,-55.524002,20.532688,4.088299,-55.520350,20.533790,3.880643,...,6.759408,9.373836,69.164310,1.714536,-8.981655,60.798400,2.208715,12.073682,65.013010,-3.638816
812,0.671323,-52.080640,47.075195,0.136766,-54.549760,42.375120,-0.409408,-54.510723,42.376280,-0.962828,...,-12.525895,3.112711,56.147213,-14.959841,-1.788000,61.841210,-11.192141,3.337535,65.352820,-15.051384


**Transforming the data**

In [18]:
xyz_transformer = XYZ_Data_Transformer()


xyz_transformer.fit(X_train)
X_train = xyz_transformer.transform()


pose_transformer = LabelEncoder()
y_train = pose_transformer.fit_transform(y_train)


Data shape before
(767, 99)
Magnitutes shape
(767, 33)
SHape after concat
(767, 132)


In [19]:
X_train.head(3)

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,left_hip,right_hip,left_knee,right_knee,left_ankle,right_ankle,left_heel,right_heel,left_foot_index,right_foot_index
0,-0.067530,2.609996,0.695646,-0.038407,2.716032,0.599378,-0.024285,2.730194,0.599535,-0.009992,...,-0.660919,-0.660918,0.321671,-0.035988,-0.072182,0.050154,-0.091523,0.047781,-0.121370,0.082268
1,-1.270197,2.805146,-1.616569,-1.385534,2.683046,-1.655619,-1.488174,2.475601,-1.655833,-1.515674,...,-0.038447,-0.038447,0.430348,0.709815,0.587575,0.803416,0.591281,0.824400,0.406357,0.741295
2,-0.021942,-0.673461,-0.186488,0.010227,-0.666082,-0.142685,0.024658,-0.661795,-0.142886,0.038705,...,-0.996702,-0.996702,-0.291063,-0.314027,-0.176953,-0.204339,-0.168334,-0.203400,-0.252032,-0.302733


In [20]:
X_train.columns

Index(['x_nose', 'y_nose', 'z_nose', 'x_left_eye_inner', 'y_left_eye_inner',
       'z_left_eye_inner', 'x_left_eye', 'y_left_eye', 'z_left_eye',
       'x_left_eye_outer',
       ...
       'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle',
       'right_ankle', 'left_heel', 'right_heel', 'left_foot_index',
       'right_foot_index'],
      dtype='object', length=132)

In [21]:
y_train

array([3, 4, 0, 9, 3, 8, 3, 1, 0, 0, 4, 4, 6, 8, 4, 5, 0, 6, 2, 0, 5, 8,
       5, 8, 4, 8, 3, 7, 1, 1, 0, 3, 0, 3, 6, 6, 0, 2, 2, 1, 0, 5, 2, 8,
       1, 3, 2, 5, 2, 8, 0, 8, 5, 3, 2, 4, 8, 2, 6, 5, 1, 7, 0, 0, 9, 7,
       0, 0, 3, 9, 7, 4, 6, 9, 9, 0, 1, 0, 0, 2, 0, 0, 9, 9, 7, 4, 3, 2,
       5, 5, 9, 8, 0, 4, 3, 0, 3, 9, 4, 5, 5, 1, 0, 4, 6, 2, 0, 2, 5, 3,
       9, 9, 0, 1, 0, 3, 4, 1, 7, 1, 4, 4, 1, 3, 0, 4, 7, 1, 0, 0, 9, 5,
       0, 0, 4, 5, 3, 3, 7, 1, 2, 2, 9, 9, 1, 1, 2, 7, 9, 8, 3, 8, 2, 0,
       9, 1, 8, 8, 1, 1, 6, 1, 9, 7, 2, 3, 1, 6, 6, 2, 2, 5, 6, 1, 1, 2,
       6, 5, 9, 7, 9, 2, 4, 1, 3, 1, 5, 4, 3, 1, 8, 8, 1, 2, 6, 1, 8, 1,
       9, 1, 8, 4, 2, 0, 3, 3, 4, 5, 9, 9, 4, 3, 3, 1, 6, 4, 0, 7, 9, 5,
       9, 8, 4, 7, 6, 1, 9, 5, 4, 5, 0, 6, 4, 1, 5, 6, 8, 7, 4, 2, 0, 4,
       5, 2, 8, 3, 4, 1, 1, 9, 9, 7, 1, 0, 5, 5, 8, 5, 6, 9, 3, 6, 5, 2,
       3, 5, 6, 3, 0, 0, 7, 5, 1, 2, 4, 8, 1, 1, 5, 0, 7, 6, 2, 6, 6, 8,
       0, 1, 3, 6, 2, 8, 3, 6, 9, 1, 4, 2, 2, 2, 7,

In [22]:
X_train.shape

(767, 132)

In [23]:
y_train.shape

(767,)

# **Creating a Random Forest model**

In [24]:
rf = RandomForestClassifier(n_estimators = 10000, random_state = 42)


rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=10000, random_state=42)

**Transforming the test data**

In [25]:
X_test = xyz_transformer.fit(X_test).transform()
y_test = pose_transformer.fit_transform(y_test)

Data shape before
(330, 99)
Magnitutes shape
(330, 33)
SHape after concat
(330, 132)


**Predicting**

In [26]:
y_pred=rf.predict(X_test)

print(y_pred.shape, y_test.shape)
print("Accuracy:", accuracy_score(y_test, y_pred))
print('Confusion matrix: \n', confusion_matrix(y_test, y_pred))
print('classification report: \n', classification_report(y_test, y_pred))

(330,) (330,)
Accuracy: 0.803030303030303
Confusion matrix: 
 [[42  0  0  1  0  0  0  0  0  3]
 [ 0 28  5  0  0  0  0  1  0  6]
 [ 0  1 35  0  1  0  1  0  0  0]
 [ 0  3  1 23  0  1  0  2  0  1]
 [ 0  0  0  0 14  4  2  0  0  0]
 [ 0  0  0  0  2 34  1  0  0  0]
 [ 0  0  0  0  1  0 15  0  0  0]
 [ 0  0  3  0  1  0  2 20  3  0]
 [ 0  2  0  2  0  0  1  2 30  0]
 [ 4  4  1  3  0  0  0  0  0 24]]
classification report: 
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        46
           1       0.74      0.70      0.72        40
           2       0.78      0.92      0.84        38
           3       0.79      0.74      0.77        31
           4       0.74      0.70      0.72        20
           5       0.87      0.92      0.89        37
           6       0.68      0.94      0.79        16
           7       0.80      0.69      0.74        29
           8       0.91      0.81      0.86        37
           9       0.71      0.67      0.

# GridSearch

In [27]:
class_weights = {}
for idx, x in enumerate(compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)):
    class_weights[idx] = x


In [28]:
models = {
    'Random Forest': RandomForestClassifier(class_weight=class_weights),
    'Gradient Boosting': GradientBoostingClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'SVC': SVC(class_weight=class_weights),
    "XGboost": XGBClassifier()
}
models

{'Random Forest': RandomForestClassifier(class_weight={0: 0.7264900662251655,
                                      1: 0.756551724137931, 2: 0.891869918699187,
                                      3: 1.0157407407407408,
                                      4: 1.3378048780487806,
                                      5: 0.9539130434782609,
                                      6: 1.3378048780487806,
                                      7: 1.388607594936709,
                                      8: 1.0861386138613862,
                                      9: 0.9882882882882883}),
 'Gradient Boosting': GradientBoostingClassifier(),
 'KNeighbors': KNeighborsClassifier(),
 'SVC': SVC(class_weight={0: 0.7264900662251655, 1: 0.756551724137931,
                   2: 0.891869918699187, 3: 1.0157407407407408,
                   4: 1.3378048780487806, 5: 0.9539130434782609,
                   6: 1.3378048780487806, 7: 1.388607594936709,
                   8: 1.0861386138613862, 9: 0.9882882882

In [29]:
for key, model in models.items():
    scores = cross_val_score(model, X_train, y_train)
    print(key, ' scores: ', scores)
    print(key, ' mean: ', scores.mean())

Random Forest  scores:  [0.83766234 0.81818182 0.83660131 0.78431373 0.84313725]
Random Forest  mean:  0.823979288685171
Gradient Boosting  scores:  [0.87012987 0.80519481 0.85620915 0.76470588 0.79738562]
Gradient Boosting  mean:  0.8187250657838894
KNeighbors  scores:  [0.78571429 0.81168831 0.76470588 0.77777778 0.75816993]
KNeighbors  mean:  0.7796112384347678
SVC  scores:  [0.78571429 0.78571429 0.74509804 0.73856209 0.75816993]
SVC  mean:  0.7626517273576097
XGboost  scores:  [0.83766234 0.82467532 0.83006536 0.78431373 0.82352941]
XGboost  mean:  0.8200492318139376


Looks like Random Forest is preferred model. We'll optimize it:

In [30]:
optimizedForest = GridSearchCV(RandomForestClassifier(class_weight=class_weights), {
    'n_estimators': [50, 100, 150, 200],
    'criterion': ['gini', 'entropy']
})
optimizedForest.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(class_weight={0: 0.7264900662251655,
                                                            1: 0.756551724137931,
                                                            2: 0.891869918699187,
                                                            3: 1.0157407407407408,
                                                            4: 1.3378048780487806,
                                                            5: 0.9539130434782609,
                                                            6: 1.3378048780487806,
                                                            7: 1.388607594936709,
                                                            8: 1.0861386138613862,
                                                            9: 0.9882882882882883}),
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [50, 100, 150, 200]})

In [31]:
results = pd.DataFrame(optimizedForest.cv_results_)
print(results[['param_n_estimators', 'param_criterion', 'mean_test_score']])
print('Best Sccore: ', optimizedForest.best_score_)
print('Optimal Param: ', optimizedForest.best_params_)

  param_n_estimators param_criterion  mean_test_score
0                 50            gini         0.816179
1                100            gini         0.825253
2                150            gini         0.825261
3                200            gini         0.823988
4                 50         entropy         0.812240
5                100         entropy         0.831780
6                150         entropy         0.823962
7                200         entropy         0.825286
Best Sccore:  0.8317799847211612
Optimal Param:  {'criterion': 'entropy', 'n_estimators': 100}


Now let's train a model with best performance parameters

In [32]:
optimized_model = RandomForestClassifier(class_weight=class_weights, criterion='gini', n_estimators=150)
optimized_model.fit(X_train, y_train)
optimized_model.score(X_test, y_test)

0.8090909090909091

In [33]:
y_predicted = optimized_model.predict(X_test)
print(classification_report(y_test, y_predicted, target_names=pose_transformer.classes_))

                    precision    recall  f1-score   support

jumping_jacks_down       0.95      0.91      0.93        46
  jumping_jacks_up       0.72      0.72      0.73        40
      pullups_down       0.79      0.89      0.84        38
        pullups_up       0.82      0.74      0.78        31
      pushups_down       0.72      0.65      0.68        20
        pushups_up       0.87      0.92      0.89        37
        situp_down       0.60      0.94      0.73        16
          situp_up       0.87      0.69      0.77        29
       squats_down       0.91      0.84      0.87        37
         squats_up       0.72      0.72      0.72        36

          accuracy                           0.81       330
         macro avg       0.80      0.80      0.80       330
      weighted avg       0.82      0.81      0.81       330

